In [ ]:
# Import python packages
import os
import warnings
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import IFrame, display, Markdown
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.patches import Rectangle
from astropy.visualization import ZScaleInterval

%matplotlib inline

In [ ]:
# Import LSST Science Pipelines packages (see pipelines.lsst.io)
import lsst.daf.base as dafBase
from lsst.daf.butler import Butler
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

# Use lsst.afw.display with the matplotlib backend
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
# Define the dataId
dataId = {'band': 'i', 'visit': 512055, 'detector': 75}

# For DC2 gen3, these are the only optoins
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'

# Use the butler to get the calexp
butler = Butler(repo, collections=collection)
registry = butler.registry

registry.expandDataId(dataId)
calexp = butler.get('calexp', dataId)


In [ ]:
import lsst.pipe.tasks.fakes as fakes

In [ ]:
from collections import namedtuple
positionTuple = namedtuple("positionTuple", "y x")

# Ground truth position and intensities for the fake sources
fakeSources = [(positionTuple(800, 455), 11342),
               (positionTuple(800, 350), 18235),
               (positionTuple(834, 379), 13574),
               (positionTuple(834, 442), 12456)]


In [ ]:
orig_calexp = calexp.clone()

Using this as an example: https://github.com/lsst/pipe_tasks/blob/387f8f07a2b66205f9fa6bda9a89dcdbbef3f64c/tests/test_fakeProcessing.py

In [ ]:
# make stars at a given position with a given intensity
def makeFakeStar(position, intensity, psf):
    psfImage = psf.computeImage(geom.Point2D(position.x, position.y)).getArray()
    psfImage *= intensity
    noise = np.random.normal(0, np.sqrt(abs(psfImage)))
    return psfImage + noise, noise

In [ ]:
# Fetch objects from the exposure
psf = calexp.getPsf()
image = calexp.getMaskedImage().getImage()
mask = calexp.getMaskedImage().getMask()
variance = calexp.getMaskedImage().getVariance()

y0 = image.getY0()
x0 = image.getX0()

# Bitplane to set corresponding to the FAKE bit
# fakeMaskValue = 2**mask.getMaskPlaneDict()['FAKE']
fakeMaskValue = 2**12

# At each position create a star with the given intensity and add it
# to the image.
for pos, intensity in fakeSources:
    objArray, noiseArray = makeFakeStar(pos, intensity, psf)
    psfRad = int((objArray.shape[0]-1)/2.)
    yslice, xslice = slice(pos.y-psfRad-y0, pos.y+psfRad+y0+1),\
                     slice(pos.x-psfRad-x0, pos.x+psfRad+x0+1)

    image.getArray()[yslice, xslice] += objArray
    mask.getArray()[yslice, xslice] += fakeMaskValue
    variance.getArray()[yslice, xslice] += noiseArray**2



In [ ]:
image0 = image.clone()
print(np.std(image0.getArray()))
image0.getArray()[yslice, xslice] += 5*objArray
print(np.std(image0.getArray()))


In [ ]:
a = image.getArray()[yslice, xslice]
print(np.std(a), np.shape(a))
a += objArray
print(np.std(a), np.shape(a))


In [ ]:
xslice

In [ ]:
diff = image.clone()
diff -= orig_calexp.image

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 7))
plt.sca(ax[0])  # set the first axis as current
display1 = afwDisplay.Display(frame=fig)
display1.scale('linear', 'zscale')
display1.mtv(orig_calexp.image)
plt.title('original image')
plt.ylim(700, 900)
plt.xlim(300, 500)
plt.sca(ax[1])  # set the second axis as current
display2 = afwDisplay.Display(frame=fig)
display2.scale('linear', 'zscale')
display2.mtv(calexp.image)
# display2.mtv(image0)
plt.title('with fake star')
plt.ylim(700, 900)
plt.xlim(300, 500)
plt.tight_layout()
plt.show()
# remove_figure(fig)

In [ ]:
# Import tasks from the LSST Science Pipelines
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.pipe.tasks.calibrate import CalibrateTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask

In [ ]:
# Create a basic schema to use with these tasks
schema = afwTable.SourceTable.makeMinimalSchema()
print(schema)

# Create a container which will be used to record metadata about algorithm execution
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

In [ ]:
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask = CharacterizeImageTask(None, config=config)

config = SourceDetectionTask.ConfigClass()
config.thresholdValue = 10      # detection threshold in units of thresholdType
config.thresholdType = "stdev"   # units for thresholdValue
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)
sourceDeblendTask = SourceDeblendTask(schema=schema)

config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config, algMetadata=algMetadata)

In [ ]:
tab = afwTable.SourceTable.make(schema)

In [ ]:
# Image characterization (this cell may take a few seconds)
result = charImageTask.run(calexp)

psf = calexp.getPsf()
sigma = psf.computeShape().getDeterminantRadius()
pixelScale = calexp.getWcs().getPixelScale().asArcseconds()

# The factor of 2.355 converts from std to fwhm
print('psf fwhm = {:.2f} arcsec'.format(sigma*pixelScale*2.355))

In [ ]:
# Source detection (this cell may take a few seconds)
result = sourceDetectionTask.run(tab, calexp)
type(result)

In [ ]:
for k, v in result.getDict().items():
    print(k, type(v))

In [ ]:
sources = result.sources

In [ ]:
# Source deblending
sourceDeblendTask.run(calexp, sources)

# Source measurement (catch future warning about machine precision)
sourceMeasurementTask.run(measCat=sources, exposure=calexp)

In [ ]:
# The copy makes sure that the sources are sequential in memory
sources = sources.copy(True)

# Investigate the output source catalog
sources.asAstropy()

In [ ]:
near_fakes = (np.abs(sources['base_SdssCentroid_y'] - 800) < 100) & (np.abs(sources['base_SdssCentroid_x'] - 400) < 100)

In [ ]:
aaa = sources[near_fakes].copy(deep=True)
len(aaa)

In [ ]:
print(aaa['base_SdssCentroid_x'], '\n', aaa['slot_PsfFlux_instFlux'])
print(aaa['base_SdssCentroid_y'])
print(fakeSources)

In [ ]:
from lsst.pipe.tasks.processCcd import ProcessCcdTask
import lsst.pipe.tasks.processCcd as processCcd

In [ ]:
# Set ouput path and create a dataId
OutputName = 'temp'
outPath = tempfile.mkdtemp() if OutputName is None \
                             else "{}-ProcessCcd".format(OutputName)
dataId = dict(visit=1)
dataIdStrList = ["{}={}".format(*item) for item in dataId.items()]
mask = None
maskPlaneName = "FAKE" 

In [ ]:
processCcdConfig = processCcd.ProcessCcdConfig
processCcdConfig.calibrate.doInsertFakes = True

In [ ]:
processCcdConfig = ProcessCcdTask.ConfigClass()
processCcdConfig.calibrate.doInsertFakes = True
processCcdTask = ProcessCcdTask(None, config=processCcdConfig)

In [ ]:
ProcessCcdTask?